# Writing Photon-HDF5 files in python

> This notebook shows how to create a [Photon-HDF5](http://photon-hdf5.org) file
> from scratch using dummy data and [phconvert](http://photon-hdf5.github.io/phconvert/).
> 
> For more info see [Writing Photon-HDF5 files](http://photon-hdf5.readthedocs.org/en/latest/writing.html).

In [ ]:
import phconvert as phc
phc.__version__

# 1. Create some dummy data

In [ ]:
import numpy as np

In [ ]:
timestamps = np.random.randint(low=0, high=2e8, size=10000).astype('int64')
timestamps.sort()
timestamps_unit = 10e-9  # 10 ns, units are always S.I.

In [ ]:
detectors = np.random.randint(low=0, high=1, size=10000).astype('uint8')

# 2. Create some metadata

In [ ]:
description = 'This is a fake dataset which mimics smFRET data.'

author = 'Author Name'
author_affiliation = 'Name of Research Institution'

sample_name = 'describe the sample here'
buffer_name = 'describe the buffer here'
dye_names = 'Cy3B, ATTO647N'   # Comma separates names of fluorophores

# 3. Create Photon-HDF5 data structure

In this section we create all the mandatory and non mandatory groups.
Not all of the are required to save a valid Photon-HDF5 file
(see example in section 4).

## 3.1 `photon_data` group

Contains arrays of photon-data: timestamps, detectors, nanotimes, etc...

*See [photon_data group reference](http://photon-hdf5.readthedocs.org/en/latest/phdata.html#photon-data-group)*

In [ ]:
photon_data = dict(
    timestamps=timestamps,
    detectors=detectors,
    timestamps_specs={'timestamps_unit': timestamps_unit})

## 3.2 `setup` group

The `/setup` group contains information about the measurement setup. 

*See [setup group reference](http://photon-hdf5.readthedocs.org/en/latest/phdata.html#setup-group).*

In [ ]:
setup = dict(
    ## Mandatory fields
    num_pixels = 2,                   # using 2 detectors
    num_spots = 1,                    # a single confoca excitation
    num_spectral_ch = 2,              # donor and acceptor detection 
    num_polarization_ch = 1,          # no polarization selection 
    num_split_ch = 1,                 # no beam splitter
    modulated_excitation = False,     # CW excitation, no modulation 
    excitation_alternated = [False],  # CW excitation, no modulation 
    lifetime = False,                 # no TCSPC in detection
    
    ## Optional fields
    excitation_wavelengths = [532e-9],         # List of excitation wavelenghts
    excitation_cw = [True],                    # List of booleans, True if wavelength is CW
    detection_wavelengths = [580e-9, 640e-9],  # Nominal center wavelength 
                                               # each for detection ch
)

## 3.3 `provenance` group

Non-mandatory group containing info about the original file 
prior to Photon-HDF5 conversion. If some information is not 
available the relative field may be omitted.

*See [provenance group documentation](http://photon-hdf5.readthedocs.org/en/latest/phdata.html#provenance-group).*

In [ ]:
provenance = dict(
    filename='original_data_file.dat', 
    software='Acquisition Software Name')

## 3.4 `identity` group

Non-mandatory group containing info about information 
this specific Photon-HDF5 file.

*See [identity group documentation](http://photon-hdf5.readthedocs.org/en/latest/phdata.html#identity-group).*

In [ ]:
identity = dict(
    author=author,
    author_affiliation=author_affiliation)

## 3.5 `measurement_specs` group

The optional /photon_data/measurement_specs group contains 
additional information allowing unambiguous interpretation 
of the data for each specific type of measurement.

*See [measurement_specs group documentation](http://photon-hdf5.readthedocs.org/en/latest/phdata.html#measurement-specs).*

In [ ]:
measurement_specs = dict(
    measurement_type = 'smFRET',
    detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                       'spectral_ch2': [1]}  # list of acceptor's detector IDs
    )

# 4. Save Photon-HDF5 files

To save a file we need to join together the root fields and group 
in a single dictionary. Here we provide a few examples.

## 4.1 Minimal file

Create a bare-bone Photon-HDF5 file with only mandatory fields.

In [ ]:
data = dict(
    description=description,
    photon_data = photon_data,
    setup=setup,
)

In [ ]:
phc.hdf5.save_photon_hdf5(data, h5_fname='dummy_dataset_barebone.h5', overwrite=True)

> **NOTE:** a user of this file can read the data but does not know
> what kind of measurement it is (e.g. smFRET with single laser
> excitation and 2-colors detection).

## 4.2 Expanded Photon-HDF5

Create a Photon-HDF5 with non-mandatory fields (including `measurement_specs`):

In [ ]:
photon_data['measurement_specs'] = measurement_specs

data = dict(
    description=description,
    photon_data = photon_data,
    setup=setup,
    identity=identity,
    provenance=provenance
)

In [ ]:
phc.hdf5.save_photon_hdf5(data, h5_fname='dummy_dataset_complete.h5', overwrite=True)

> **NOTE:** a user of this file can correctly interpret the data
> reading that the measurement type is 'smFRET' (meaning smFRET with single laser
> excitation and 2-colors detection) and the IDs of donor and acceptor detectors
> (from `detectors_specs/spectral_ch1` and `spectral_ch2` respectively).